* Complete Linkage Criterion - Distance between two clusters is the largest distance between all possible pairs of points chosen from the clusters
* Divisive Clustering - Initially, all the data points are put into one cluster and then the cluster is divided into smaller clusters based upon the linkage criterion

# Imports

In [93]:
import csv
import numpy as np
import time

In [94]:
# These mappings are for string type data which cannot be
# Converted to integers while reading the rows as numpy arrays
genders = {
    'Male': 0,
    'Female': 1
}

custType = {
    'disloyal Customer': 0,
    'Loyal Customer': 1
}

typeTravel = {
    'Personal Travel': 0,
    'Business travel': 1
}

travelClass = {
    'Eco': 0,
    'Business': 1,
    'Eco Plus': 2
}

file_format_string = 'kmeans_{}.txt'

# Functions and Classes

In [95]:
# Function to read the csv file and return a 2D numpy array
def read_data(data_file):
    data = []

    with open(data_file, 'r') as f:
        reader = csv.DictReader(f)

        for row in reader:
            try:
                # Store the data as a numpy array
                data.append([
                    int(row['id']),
                    genders[row['Gender']],
                    custType[row['Customer Type']],
                    int(row['Age']),
                    typeTravel[row['Type of Travel']],
                    travelClass[row['Class']],
                    int(row['Flight Distance']),
                    int(row['Inflight wifi service']),
                    int(row['Departure/Arrival time convenient']),
                    int(row['Ease of Online booking']),
                    int(row['Gate location']),
                    int(row['Food and drink']),
                    int(row['Online boarding']),
                    int(row['Seat comfort']),
                    int(row['Inflight entertainment']),
                    int(row['On-board service']),
                    int(row['Leg room service']),
                    int(row['Baggage handling']),
                    int(row['Checkin service']),
                    int(row['Inflight service']),
                    int(row['Cleanliness']),
                    int(row['Departure Delay in Minutes']),
                    # This is done since there were a few instances when Arrival Delay in Minutes was empty
                    (int(row['Arrival Delay in Minutes']) if row['Arrival Delay in Minutes'] else 0),
                ])
            except Exception as e:
                print(e)
                print(row)
                break

    data = np.array(data)

    return data

# Function to retrieve clusters written onto a file
def read_cluster_data(cluster_file):
    clusters = []

    with open(cluster_file, 'r') as f:
        lines = f.readlines()

        # Since each line is a cluster, we add this line to
        # clusters after splitting by ','
        for line in lines:
            indices = line.split(',')

            clusters.append([int(i) for i in indices])

    return clusters

In [96]:
# Function to write clusters information to output file
def save_cluster_info(clusters, k, output_file):
    # Clearing the output file
    print('', end='', file=open(output_file, 'w'))

    # For each cluster, printing the indices of the data points in that cluster

    # Had to do this check since numpy gave issues when deleting and adding
    # Clusters in divisive clustering, so a simple array is returned there
    if isinstance(clusters, np.ndarray):
        for i in range(k):
            inds = np.where(clusters == i)[0]
            print(",".join(str(ind) for ind in inds), file=open(output_file, 'a'))
    else:
        for cluster in clusters:
            print(",".join(str(ind) for ind in cluster), file=open(output_file, 'a'))

In [97]:
# cosine similarity = A.B/|A||B|
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

# Mean distance from the sample at i to all other data points in the cluster
def mean_intra_cluster_distance(i, data, clusters, cluster):
    cluster_data = data[clusters == cluster]
    data_point = data[i]

    distances = [1 - cosine_similarity(data_point, point) for point in cluster_data]
    return np.mean(distances)

# Mean distance from the sample to all other data points in the nearest cluster
def mean_inter_cluster_distance(i, data, clusters, cluster):
    # Distance to each cluster
    cluster_distances = []
    data_point = data[i]

    k = len(np.unique(clusters))

    for j in range(k):
        if j == cluster:
            continue

        current_cluster_data = data[clusters == j]

        distances = np.array([1 - cosine_similarity(data_point, point) for point in current_cluster_data])

        cluster_distances.append(np.mean(distances))

    # return the minimum distance to the other clusters
    return np.min(cluster_distances)

# Silhouette coefficient = inter - intra / max(intra, inter)
def silhouette_coefficient(i, data, clusters):
    cluster = clusters[i]

    a = mean_intra_cluster_distance(i, data, clusters, cluster)
    b = mean_inter_cluster_distance(i, data, clusters, cluster)

    return (b - a) / max(a, b)

# Final score is the mean of scores of all samples
def silhouette_score(data, clusters):
    scores = [silhouette_coefficient(i, data, clusters) for i in range(len(data))]
    return np.mean(scores)

# Assign data points to clusters depending on which
# centroid it is most similar(closest) to
def assign_clusters(data, centroids):
    clusters = []

    for point in data:
        similarities = [cosine_similarity(point, centroid) for centroid in centroids]
        cluster = np.argmax(similarities)
        clusters.append(cluster)

    return np.array(clusters)

# After assigning points to clusters, we find the new
# centroids of each cluster
def update_centroids(data, clusters, k):
    centroids = []

    for i in range(k):
        cluster_data = data[clusters == i]
        centroid = np.mean(cluster_data, axis=0)
        centroids.append(centroid)

    return np.array(centroids)

# We put data points in each cluster based on
# their distance to the centroids
def k_means_clustering(data, k, iterations=20):
    # Randomly initialize centroids
    np.random.seed(0)
    centroids = data[np.random.choice(data.shape[0], k, replace=False)]

    for _ in range(iterations):
        # In each iteration, assign clusters and update centroids
        clusters = assign_clusters(data, centroids)
        centroids = update_centroids(data, clusters, k)

    # Write final cluster information to a file named kmeans_k.txt
    save_cluster_info(clusters, k, file_format_string.format(k))

    return clusters

In [98]:
# Function to get the distance matrix
# Distance is 1 - cos(a, b)
def get_distance_matrix(data):
    n = len(data)
    dist_mat = np.zeros((n, n))

    for i in range(n):
        for j in range(i+1, n):
            dist_mat[i][j] = 1 - cosine_similarity(data[i], data[j])
            dist_mat[j][i] = dist_mat[i][j]

    return dist_mat

# Function to choose the cluster that we split at the current iteration
# Here we choose the larger of the two clusters that are the farthest away
def choose_cluster(distance_matrix, clusters):
    max_dist = 0
    max_i = -1
    max_j = -1

    for i in range(len(clusters)):
        for j in range(i+1, len(clusters)):
            cluster_i = clusters[i]
            cluster_j = clusters[j]
            # Distance between clusters is according to complete linkage
            dist = np.max(distance_matrix[cluster_i][:, cluster_j])
            if dist > max_dist:
                max_dist = dist
                max_i = i
                max_j = j

    if max_i == -1:
        return clusters[0], 0

    # Between max_i and max_j choose the larger cluster
    if len(clusters[max_i]) > len(clusters[max_j]):
        return clusters[max_i], max_i
    else:
        return clusters[max_j], max_j

# Top down divisive clustering
def divisive_clustering(data, k):
    dist_mat = get_distance_matrix(data)

    # Initially all data points are in the same cluster
    clusters = [[i for i in range(len(data))]]

    while len(clusters) < k:
        # Choose a cluster and the pair of points with the maximum distance
        cluster, c = choose_cluster(dist_mat, clusters)

        # Divide this cluster into two using a flat clustering subroutine
        # We will use k-means with k=2 to divide the cluster
        cluster_data = data[cluster]
        cluster_clusters = k_means_clustering(cluster_data, 2)

        # Update the clusters
        new_clusters = []
        for i in range(2):
            new_cluster = [cluster[j] for j in range(len(cluster)) if cluster_clusters[j] == i]
            new_clusters.append(new_cluster)

        # Remove the old cluster and add the new clusters
        clusters.pop(c)
        clusters.extend(new_clusters)

    # Write final cluster information to a file named divisive.txt
    save_cluster_info(clusters, k, './divisive.txt')

    return clusters

# Jaccard similarity of two sets is the length of their intersection divided by the length of their union
def jaccard_similarity(cluster1, cluster2):
    intersection = len(np.intersect1d(cluster1, cluster2))
    union = len(np.union1d(cluster1, cluster2))
    return intersection / union

# Main

## Data Reading

In [99]:
data_file = './airpass.csv'
kmeans_file = './kmeans.txt'

In [100]:
data = read_data(data_file)

## Finding Optimal K

In [101]:
sil_scores = []

### K = 3

In [102]:
start = time.time()
clusters_3 = k_means_clustering(data, 3)
end = time.time()

print('Time taken for clustering k = 3:', end - start)

start = time.time()
sil_scores.append(silhouette_score(data, clusters_3))
end = time.time()

print('Silhouette score for k = 3:', sil_scores[-1])
print('Time taken for silhouette score calculation for k = 3:', end - start)

Time taken for clustering k = 3: 2.9101600646972656
Silhouette score for k = 3: 0.627480508197062
Time taken for silhouette score calculation for k = 3: 102.20136094093323


### K = 4

In [103]:
start = time.time()
clusters_4 = k_means_clustering(data, 4)
end = time.time()

print(f'Time taken for clustering k = 4: {end - start}s')

start = time.time()
sil_scores.append(silhouette_score(data, clusters_4))
end = time.time()

print(f'Silhouette score for k = 4: {sil_scores[-1]}')
print(f'Time taken for silhouette score calculation for k = 4: {end - start}s')

Time taken for clustering k = 4: 2.646796703338623s
Silhouette score for k = 4: 0.6331171650201574
Time taken for silhouette score calculation for k = 4: 99.39695525169373s


### K = 5

In [104]:
start = time.time()
clusters_5 = k_means_clustering(data, 5)
end = time.time()

print(f'Time taken for clustering k = 5: {end - start}s')

start = time.time()
sil_scores.append(silhouette_score(data, clusters_5))
end = time.time()

print(f'Silhouette score for k = 5: {sil_scores[-1]}')
print(f'Time taken for silhouette score calculation for k = 5: {end - start}s')

Time taken for clustering k = 5: 3.2327077388763428s
Silhouette score for k = 5: 0.558882776386343
Time taken for silhouette score calculation for k = 5: 98.14795994758606s


### K = 6

In [105]:
start = time.time()
clusters_6 = k_means_clustering(data, 6)
end = time.time()

print(f'Time taken for clustering k = 6: {end - start}s')

start = time.time()
sil_scores.append(silhouette_score(data, clusters_6))
end = time.time()

print(f'Silhouette score for k = 6: {sil_scores[-1]}')
print(f'Time taken for silhouette score calculation for k = 6: {end - start}s')


Time taken for clustering k = 6: 3.7355403900146484s
Silhouette score for k = 6: 0.5229627147356585
Time taken for silhouette score calculation for k = 6: 100.0646448135376s


In [106]:
# K value with highest silhouette score
best_k = np.argmax(sil_scores) + 3

print(f"Best K value: {best_k}")

best_k_clusters = read_cluster_data(file_format_string.format(best_k))


Best K value: 4


In [107]:
# take the file kmeans_{best_k}.txt and copy the text to kmeans.txt
print('', end='', file=open(kmeans_file, 'w'))
with open(file_format_string.format(best_k), 'r') as f:
    for line in f:
        print(line, end='', file=open(kmeans_file, 'a'))

## Heirarchal Clustering

In [108]:
start = time.time()
div_clusters = divisive_clustering(data, best_k)
end = time.time()

print(f'Time taken for divisive clustering: {end - start}s')

Time taken for divisive clustering: 59.84216022491455s


In [109]:
print(div_clusters)

[[14, 55, 60, 77, 82, 109, 117, 135, 145, 167, 193, 206, 207, 212, 222, 233, 239, 240, 274, 303, 317, 331, 334, 344, 378, 427, 434, 477, 496, 503, 517, 525, 532, 545, 552, 568, 675, 681, 701, 704, 740, 789, 842, 851, 853, 869, 872, 885, 909, 968, 971, 994, 997, 1003, 1020, 1034, 1116, 1130, 1136, 1155, 1166, 1168, 1200, 1205, 1231, 1234, 1244, 1245, 1272, 1283, 1284, 1291, 1326, 1353, 1365, 1385, 1397, 1412, 1423, 1451, 1455, 1465, 1493, 1524, 1539, 1546, 1547, 1565, 1588, 1595, 1605, 1613, 1635, 1675, 1683, 1694, 1705, 1754, 1758, 1761, 1766, 1785, 1811, 1817, 1824, 1839, 1858, 1895, 1897, 1930, 1961, 1980, 1991, 2039, 2075, 2082, 2084, 2104, 2109, 2113, 2138, 2142, 2147, 2162, 2169, 2172, 2189, 2256, 2262, 2275, 2281, 2282, 2330, 2352, 2354, 2378, 2427, 2432, 2435, 2516, 2517, 2527, 2562, 2565, 2580, 2585, 2663, 2665, 2671, 2677, 2680, 2698, 2750, 2767, 2846, 2858, 2887, 2916, 2928, 2939, 2954, 2958, 2959, 2969, 2982, 2991, 2994], [1, 3, 4, 6, 9, 17, 19, 23, 25, 26, 27, 37, 41, 48, 6

## Mapping the clusters

In [110]:
# Computing the Jaccard similarity matrix
similarity_matrix = np.zeros((best_k, best_k))
for i in range(best_k):
    for j in range(best_k):
        clusterA = best_k_clusters[i]
        clusterB = div_clusters[j]

        similarity_matrix[i][j] = jaccard_similarity(clusterA, clusterB)

print('Similarity matrix:')
print(similarity_matrix)

Similarity matrix:
[[0.48502994 0.         0.         0.        ]
 [0.         0.         0.22111056 0.72461059]
 [0.20427553 0.27608696 0.         0.        ]
 [0.         0.4723127  0.27824859 0.        ]]


In [112]:
# Now we map the div_clusters to the best_k_clusters
mapped_clusters = []

for i in range(best_k):
    max_similarity = 0
    max_j = -1
    for j in range(best_k):
        if similarity_matrix[i][j] > max_similarity and j not in mapped_clusters:
            max_similarity = similarity_matrix[i][j]
            max_j = j
    mapped_clusters.append(max_j)


# Print the mapping and the jaccard similarity score for each cluster
for i in range(best_k):
    print(f'Cluster {i} in k-means clustering maps to cluster {mapped_clusters[i]} in divisive clustering with a Jaccard similarity score of {similarity_matrix[i][mapped_clusters[i]]}')


Cluster 0 in k-means clustering maps to cluster 0 in divisive clustering with a Jaccard similarity score of 0.48502994011976047
Cluster 1 in k-means clustering maps to cluster 3 in divisive clustering with a Jaccard similarity score of 0.7246105919003115
Cluster 2 in k-means clustering maps to cluster 1 in divisive clustering with a Jaccard similarity score of 0.27608695652173915
Cluster 3 in k-means clustering maps to cluster 2 in divisive clustering with a Jaccard similarity score of 0.2782485875706215
